In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
import tensorflow as tf
import keras.backend.tensorflow_backend as KTF
config = tf.ConfigProto()
config.gpu_options.allow_growth=True   #不全部占满显存, 按需分配
session = tf.Session(config=config)
KTF.set_session(session)
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Dropout, Embedding, Flatten, Input, concatenate
from keras.layers import Input, TimeDistributed, Dense, Lambda, concatenate, Dropout, BatchNormalization
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Nadam
import keras.backend as K
import numpy as np
import pandas as pd
import pickle


/home/kawayi-4/anaconda3/envs/python36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# line = train[:10]
# line['hour'] = line['hour'].astype(str)
# line['Source_Station_encode'] = line['Source_Station_encode'].astype('int')
# line['Target_Station_encode'] = line['Target_Station_encode'].astype('int')
# line['Source_Station_encode'] = line['Source_Station_encode'] + 1
# line['Target_Station_encode'] = line['Target_Station_encode'] + 1
# line['Source_Station_encode'] = line['Source_Station_encode'].astype('str')
# line['Target_Station_encode'] = line['Target_Station_encode'].astype('str')
# line['hour_s'] = line['hour'] + line['Source_Station_encode']
# line['hour_e'] = line['hour'] + line['Target_Station_encode']
# line['hour_s'] = line['hour_s'].astype(int)
# line['hour_e'] = line['hour_e'].astype(int)
# line

In [3]:
def zuhe(line):
    line['hour'] = line['hour'].astype(str)
    line['weekday'] = line['weekday'].astype(str)
    line['is_peek'] = line['is_peek'].astype(str)
    line['is_workday'] = line['is_workday'].astype(str)
                                   
    line['hour_weekday'] = line['hour'] + line['weekday']
    line['peak_workday'] = line['is_peek'] + line['is_workday']
    
    line['hour'] = line['hour'].astype(int)
    line['is_peek'] = line['is_peek'].astype(int)
    line['is_workday'] = line['is_workday'].astype(int)
    line['weekday'] = line['weekday'].astype(int)
    
    line['hour_weekday'] = line['hour_weekday'].astype(int)
    line['peak_workday'] = line['peak_workday'].astype(int)
    return line

In [4]:
with open('./data/train-id4-crowd-grid6.txt', 'rb') as data_file:
    train = pickle.load(data_file)
    #训练验证都划分
    #900 29 26 #800 27.9 25.9 #700 27.5 25.3 #27.4 24.8 #26.4 24.08 #23 21 #(90)250 21 19.5 #(80)17. 16.5
    #训练划分 验证不划分
    #250
train = train[(train['Diff_Time']<600)]
train = zuhe(train)
with open('./data/test-id4-crowd-grid6.txt', 'rb') as data_file:
    test = pickle.load(data_file)
test = zuhe(test)
from sklearn import preprocessing
x1 = train['ID'].as_matrix().reshape([-1,1])
N1 = x1.shape[0]

x2 = test['ID'].as_matrix().reshape([-1,1])
N2 = x2.shape[0]
x = np.concatenate((x1,x2))
x = preprocessing.LabelEncoder().fit_transform(x) #13963746x21567
print(x.shape)#(14454126, 21567)
train['new_ID'] = x[:N1]
test['new_ID'] = x[N1:]

/home/kawayi-4/anaconda3/envs/python36/lib/python3.6/site-packages/ipykernel_launcher.py:13: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  del sys.path[0]
/home/kawayi-4/anaconda3/envs/python36/lib/python3.6/site-packages/ipykernel_launcher.py:16: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  app.launch_new_instance()
/home/kawayi-4/anaconda3/envs/python36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


(14223933,)


In [5]:
col1 = ['new_ID']
col2 = [c for c in train if
       c not in ['Unnamed: 0', 'is_crowd', 'grid_aver_diff', 'grid_aver_d', 'Source_Station_encode', 'Target_Station_encode', 's_t', 'new_s_t', 's_t_o', 'new_s_t_o','ID2', 's_ij', 'e_ij', 's_x', 's_y', 'e_x', 'e_y', 'new_ID','ID','O_LINENO', 'O_UP', 'Source_Station', 'Target_Station', 'O_TIME', 'aver_v', 'max_v',
                 'Diff_Time']]
col3 = ['Source_Station_encode']
col4 = ['Target_Station_encode']
col5 = ['new_s_t']
col6 = ['new_s_t_o']

print(col1,col2,col3,col4,col5,col6)
X_train1 = train[col1].values
X_train2 = train[col2].values
X_train3 = train[col3].values
X_train4 = train[col4].values
X_train5 = train[col5].values
X_train6 = train[col6].values
y_train = train['Diff_Time'].values
print('train',X_train1.shape,X_train2.shape,X_train3.shape,X_train4.shape,X_train5.shape,X_train6.shape)

['new_ID'] ['Distance', 'hour', 'is_peek', 'weekday', 'is_workday', 'dws', 'nws', 'dts', 'nts', 'is_rain', 'h_aver_diff', 'h_aver_d', 'h_aver_v', 'hour_weekday', 'peak_workday'] ['Source_Station_encode'] ['Target_Station_encode'] ['new_s_t'] ['new_s_t_o']
train (13733553, 1) (13733553, 15) (13733553, 1) (13733553, 1) (13733553, 1) (13733553, 1)


In [6]:
col1 = ['new_ID']
col2 = [c for c in test if
       c not in ['Unnamed: 0', 'is_crowd', 'grid_aver_diff', 'grid_aver_d', 'Source_Station_encode', 'Target_Station_encode', 's_t', 'new_s_t', 's_t_o', 'new_s_t_o','ID2', 's_ij', 'e_ij', 's_x', 's_y', 'e_x', 'e_y', 'new_ID','ID','O_LINENO', 'O_UP', 'Source_Station', 'Target_Station', 'O_TIME', 'aver_v', 'max_v',
                 'Diff_Time','Distance1', 'distance2','TERMINALNO', 'new_dist']]
col3 = ['Source_Station_encode']
col4 = ['Target_Station_encode']
col5 = ['new_s_t']
col6 = ['new_s_t_o']

print(col1,col2,col3,col4,col5,col6)
X_test1 = test[col1].values
X_test2 = test[col2].values
X_test3 = test[col3].values
X_test4 = test[col4].values
X_test5 = test[col5].values
X_test6 = test[col6].values
print('test',X_test1.shape,X_test2.shape,X_test3.shape,X_test4.shape,X_test5.shape,X_test6.shape)

['new_ID'] ['Distance', 'hour', 'is_peek', 'weekday', 'is_workday', 'dws', 'nws', 'dts', 'nts', 'is_rain', 'h_aver_diff', 'h_aver_d', 'h_aver_v', 'hour_weekday', 'peak_workday'] ['Source_Station_encode'] ['Target_Station_encode'] ['new_s_t'] ['new_s_t_o']
test (490380, 1) (490380, 15) (490380, 1) (490380, 1) (490380, 1) (490380, 1)


In [7]:
from keras.layers import *
def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean((y_pred - y_true)**2))
    
def mse_loss(y_true,y_pred):
    return K.sqrt(K.mean((y_pred - y_true)**2))

def get_model():
    input1 = Input(shape=(1,))
    input2 = Input(shape=(15,))
    input3 = Input(shape=(1,))
    input4 = Input(shape=(1,))
    input5 = Input(shape=(1,))
#     input6 = Input(shape=(1,))

    x1 = Embedding(21064, 128, input_length=1)(input1)
    x1 = Flatten()(x1)
    x1 = BatchNormalization()(x1)
    
    x2 = BatchNormalization()(input2)
    x2 = Dense(units=128,activation='relu')(x2)
    
    x3 = Embedding(5772, 64, input_length=1)(input3)
    x3 = BatchNormalization()(x3)
    x3 = Flatten()(x3)
    
    x4 = Embedding(5772, 64, input_length=1)(input4)
    x4 = BatchNormalization()(x4)
    x4 = Flatten()(x4)
    
    x5 = Embedding(14144, 64, input_length=1)(input5)
    x5 = BatchNormalization()(x5)
    x5 = Flatten()(x5)
    
#     x6 = Embedding(21219, 64, input_length=1)(input6)
#     x6 = BatchNormalization()(x6)
#     x6 = Flatten()(x6)

    def dist(x):
        return x[0]*x[1]/(K.sum(x[0]**2,axis=1,keepdims=True)+K.sum(x[1]**2,axis=1,keepdims=True))    
    #     Aggregate
    x3_x4 = Lambda(dist)([x3,x4])

    x = concatenate([x1, x2, x3, x4, x5, x3_x4])
#     x = Flatten()(x)
    x = Dense(512, activation='relu')(x)
    x = Dense(512, activation='relu')(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.3)(x)
    out = Dense(1, activation='relu')(x)
    
    model = Model(inputs=[input1, input2, input3, input4, input5], outputs=out)
    model.compile(loss=mse_loss, optimizer=Nadam(), metrics=[root_mean_squared_error])
    return model



In [8]:
from sklearn.model_selection import StratifiedKFold, KFold

def get_stacking(X_train1, X_train2, X_train3, X_train4, X_train5, X_train6, labels, n_folds=10):
    train_num, test_num = X_train1.shape[0], X_test1.shape[0]
    second_level_train_set = np.zeros((train_num,))
    second_level_test_set = np.zeros((test_num,))
    test_nfolds_sets = np.zeros((test_num, n_folds))
    
    folds = KFold(n_splits=n_folds, shuffle=True, random_state=1001001)
        
    for n_fold,(train_idx, val_idx) in enumerate(folds.split(labels)):
        train_1 = X_train1[train_idx]
        train_2 = X_train2[train_idx]
        train_3 = X_train3[train_idx]
        train_4 = X_train4[train_idx]
        train_5 = X_train5[train_idx]
#         train_6 = X_train6[train_idx]
        train_y = labels[train_idx]

        val_1 = X_train1[val_idx]
        val_2 = X_train2[val_idx]
        val_3 = X_train3[val_idx]
        val_4 = X_train4[val_idx]
        val_5 = X_train5[val_idx]
#         val_6 = X_train6[val_idx]
        val_y = labels[val_idx]
#         ckpt_path = './logtianchi/'+ model_list[n_fold]
#         print(ckpt_path)
        model = get_model()
        if n_fold == 0:
            print(model.summary())
        print(n_fold)
        

        ckpt_path = './log/new_cv_'+str(n_fold)+'_weights-{val_loss:.4f}.hdf5'
        

        best_score = 1000
        best_iter = 0
        for i in range(10):
            model.fit([train_1,train_2,train_3,train_4,train_5],train_y+np.random.uniform(-1,1,(len(train_y,))), 
                      validation_data=([val_1,val_2,val_3,val_4,val_5],val_y), 
                      epochs=1, 
                      batch_size=1024)
            score = model.evaluate([val_1,val_2,val_3,val_4,val_5],val_y,batch_size=1024)[0]
            print(score)
            if score<best_score:
                best_score = score
                best_iter = i
                model.save_weights('./log/new_cv'+str(n_fold)+'.h5')
            if i-best_iter>3:
                break
        model.load_weights('./log/new_cv'+str(n_fold)+'.h5')
        second_level_train_set[val_idx] = model.predict([val_1, val_2,val_3,val_4,val_5], batch_size=1024)[:, 0]
        test_nfolds_sets[:,n_fold] = model.predict([X_test1, X_test2, X_test3, X_test4, X_test5,], batch_size=1024)[:, 0]
    
    "---save---"
    with open('./data/new_test_nfolds_sets3.txt', 'wb') as data_file:
        pickle.dump(test_nfolds_sets, data_file)

    second_level_test_set[:] = test_nfolds_sets.mean(axis=1)

    result = second_level_test_set
    
    return second_level_train_set, second_level_test_set, result

In [ ]:
train_sets = []
test_sets = []
train_set, test_set, result = get_stacking(X_train1, X_train2, X_train3, X_train4, X_train5, X_train6, y_train, n_folds=10)
train_sets.append(train_set)
test_sets.append(test_set)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 1, 64)        369408      input_3[0][0]                    
__________________________________________________________________________________________________
embedding_

In [16]:
meta_train = np.concatenate([result_set.reshape(-1,1) for result_set in train_sets], axis=1)
meta_test = np.concatenate([y_test_set.reshape(-1,1) for y_test_set in test_sets], axis=1)
print(meta_train.shape, meta_test.shape)

"---save---"
with open('./data/meta_train_nn_qrf_2.txt', 'wb') as data_file:
    pickle.dump(meta_train, data_file)
with open('./data/meta_test_nn_qrf_2.txt', 'wb') as data_file:
    pickle.dump(meta_test, data_file)

(13733553, 1) (490380, 1)


In [9]:
train_set

array([155.27383423, 225.86791992,  93.46902466, ..., 134.73045349,
       368.41989136, 125.18015289])

In [10]:
test_set

array([109.02749786, 129.785672  , 133.44480896, ...,  90.29129028,
        62.36225891, 198.12910919])

In [12]:
pred = result

In [14]:
def zuhe(line):
    line['hour'] = line['hour'].astype(str)
    line['weekday'] = line['weekday'].astype(str)
    line['s_ij'] = line['s_ij'].astype(str)
    line['e_ij'] = line['e_ij'].astype(str)

#     line['is_peek'] = line['is_peek'].astype(str)
    line['is_crowd'] = line['is_crowd'].astype(str)
                                   
    line['hour_weekday'] = line['hour'] + line['weekday']
    line['hour_crowd'] = line['hour'] + line['is_crowd']
    line['hour_s'] = line['hour'] + line['s_ij']
    line['hour_e'] = line['hour'] + line['e_ij']

    line['hour'] = line['hour'].astype(int)
#     line['is_peek'] = line['is_peek'].astype(int)
    line['is_crowd'] = line['is_crowd'].astype(int)
    line['weekday'] = line['weekday'].astype(int)
    line['s_ij'] = line['s_ij'].astype(int)
    line['e_ij'] = line['e_ij'].astype(int)
    
    line['hour_weekday'] = line['hour_weekday'].astype(int)
    line['hour_crowd'] = line['hour_crowd'].astype(int)
    line['hour_s'] = line['hour_s'].astype(int)
    line['hour_e'] = line['hour_e'].astype(int)
    return line

In [15]:
with open('./data/test-id4-crowd-grid4.txt', 'rb') as data_file:
    test = pickle.load(data_file)
test = zuhe(test)
test = test[['ID', 'new_dist',  'O_LINENO', 'O_UP', 'Source_Station', 'Target_Station',
       'Distance', 'distance2', 'O_TIME', 'hour', 'is_peek', 'weekday',
       'is_workday', 'dws', 'nws', 'dts', 'nts', 'is_rain', 'max_v',
       'h_aver_diff', 'h_aver_d', 'h_aver_v', 'TERMINALNO', 
       'is_crowd', 's_ij', 'e_ij','grid_aver_diff', 'grid_aver_d', 'hour_weekday', 'hour_crowd', 'hour_s', 'hour_e']]
test.columns=['ID', 'Distance',  'O_LINENO', 'O_UP', 'Source_Station', 'Target_Station',
       'Distance1', 'distance2', 'O_TIME', 'hour', 'is_peek', 'weekday',
       'is_workday', 'dws', 'nws', 'dts', 'nts', 'is_rain', 'max_v',
       'h_aver_diff', 'h_aver_d', 'h_aver_v', 'TERMINALNO', 
       'is_crowd', 's_ij', 'e_ij','grid_aver_diff', 'grid_aver_d', 'hour_weekday', 'hour_crowd', 'hour_s', 'hour_e']
col1 = [c for c in test if
       c not in ['Unnamed: 0','ID2' , 's_x', 's_y', 'e_x', 'e_y','O_LINENO', 'O_UP', 'Source_Station', 'Target_Station', 'O_TIME', 'aver_v', 'max_v',
                 'Diff_Time','Distance1', 'distance2','TERMINALNO', 'new_dist']]
print(col1)

test['pred1'] = pred
test['pred2'] = pred

sub1 = test[['O_LINENO','TERMINALNO', 'O_UP','Source_Station','Target_Station','O_TIME','pred1','pred2','Distance','Distance1']]
sub1['O_TIME'] = pd.to_datetime(sub1['O_TIME'],format='%Y-%m-%d %H:%M:%S')
sub1.columns = ['LINE','TERMINALNO','UP','pred_start_stop_ID','pred_end_stop_ID','realTime','pred1','pred2','Distance','Distance1']
sub1 = sub1.reset_index()
del sub1['index']

sub=pd.read_csv("./toBePredicted_0607_segment.csv", sep=",")
sub['realTime'] = pd.to_datetime(sub['realTime'],format='%Y-%m-%d %H:%M:%S')
sub2 = sub[['LINE','TERMINALNO','UP','pred_start_stop_ID','pred_end_stop_ID','realTime','distance']]
sub2=pd.merge(sub2,sub1,on=['LINE','TERMINALNO','UP','pred_start_stop_ID','pred_end_stop_ID','realTime'],how='left')


sub2['div_dist'] = sub2['Distance'] / sub2['Distance1']
sub2['new_pred'] = sub2['div_dist'] * sub2['pred1']

import math
for i in range(sub2.shape[0]):
    s = sub2.iloc[i]
    if math.isnan(s['div_dist']):
        sub2.loc[i,'new_pred'] = s['pred1']
        
sub2.to_csv('./qrf_toBePredicted_0807_result.csv',sep=",",index=False)#0605 29.2467

print('finished')

['ID', 'Distance', 'hour', 'is_peek', 'weekday', 'is_workday', 'dws', 'nws', 'dts', 'nts', 'is_rain', 'h_aver_diff', 'h_aver_d', 'h_aver_v', 'is_crowd', 's_ij', 'e_ij', 'grid_aver_diff', 'grid_aver_d', 'hour_weekday', 'hour_crowd', 'hour_s', 'hour_e']


/home/kawayi-4/anaconda3/envs/python36/lib/python3.6/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


finished


In [ ]:
!python 